In [3]:
import sys
import requests
import base64
import json
import logging
import tqdm

import pandas as pd
import numpy as np

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy.util as util

### spotify api token 받아오기

In [4]:
username = 'minseo_musiky'
client_id = 'cc46707c5e8d4d8ea3b308960dfb9c1a'
client_secret = 'client_secret'
redirect_uri = 'http://localhost:8888/callback'

#https://developer.spotify.com/documentation/general/guides/scopes/
scope = 'user-library-read user-read-playback-state playlist-read-private user-read-recently-played playlist-read-collaborative playlist-modify-public playlist-modify-private'

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
sp = spotipy.Spotify(auth = token)

### 참고사항
1. 데이터는 아래 형식에 맞춰서 해주세요.

In [8]:
track = pd.read_csv('데이터/Spotify_song/playlist/track_sample.csv', index_col=0)
track.head()

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


In [10]:
track['track_id'] = track['track_uri'].str.split(':').str[2] # track_uri로부터 track_id 뽑아내기
track['artist_id'] = track['artist_uri'].str.split(':').str[2] # artist_uri로부터 artist_id 뽑아내기
track['album_id'] = track['album_uri'].str.split(':').str[2] # album_uri로부터 album_id 뽑아내기
track.head()

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,track_id,artist_id,album_id
0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,6vV5UrXcfyQD1wu4Qo2I9K
1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,0z7pVBGOD7HCIB7S8eLkLI
2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,25hVFAxTlDvXbx2X2QkUkE
3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,6QPkyl04rXwTGlGlcYaRoW
4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,6NmFmPX56pcLBOFMhIiKvF


In [5]:
# 예시) audio_analysis 하는 방법
# 이 외의 함수들은 spotipy 홈페이지 참고할 것!!

df = pd.DataFrame() # 결과값을 저장할 데이터프레임 생성
track = track.set_index('track_id') # track 데이터프레임의 index를 track_id 컬럼으로 설정

for track_id in tqdm.tqdm_notebook(track.index): # track 데이터프레임의 index를 하나씩 받아옴
    df = pd.concat([df, pd.DataFrame(sp.audio_features(track_id))[['id','acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]]) # track_id마다 노래의 feature를 한 row씩 뽑아와서 df에 합치기.


{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1444664690,
  'analysis_time': 7.09854,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 4181536,
  'duration': 189.63882,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 5.01551,
  'start_of_fade_out': 178.06802,
  'loudness': -9.19,
  'tempo': 89.048,
  'tempo_confidence': 0.102,
  'time_signature': 3,
  'time_signature_confidence': 0.622,
  'key': 2,
  'key_confidence': 0.306,
  'mode': 0,
  'mode_confidence': 0.634,
  'codestring': 'eJw9mAmSJCkMBL-ST-CUxP8_Nu6qttk1222oBHSEIgRrvjnvvfGNL2LUjIrx7TrfjZ17xBnfe-vbs1a8ceKbx6liWb7Mb94dDjP3HPubEec7Y961YvBrLYfjnZpn89de3617Xrxb37kZnLX3WGPyXb34LqM73uKvXPXNsXJknJhfsKq3vm945mDrd1_tNyu_tw_DGJi-a33PxcmuG0vZZbMC_yLnzsc41_gSt-d7A5_m4_f78HLvO765xusxZ_jlXIvD4mXdedhuteH7FRsmTi8OblPZZq_1zb35gAmCWZulcxu1uQN